In [1]:
import itertools
import os
import glob
import pandas as pd
import pickle
from conf_dict_params import config_dict_grid, traces

In [2]:
keys, values = zip(*config_dict_grid.items())
permutation_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]
print("Len permutation dicts: ", len(permutation_dicts))

Len permutation dicts:  120


## Make a custom permutaiton dict with only best alg

In [3]:
permutation_dicts[20]

{'trace': './traces/WIRED_35mbps.json',
 'delay_states': False,
 'normalize_states': True,
 'step_time': 200,
 'alg': 'SAC',
 'tuned': False,
 'reward_profile': 0}

In [4]:
type(permutation_dicts[0]["delay_states"])

bool

In [5]:
new_permutation_dicts = []

for d in permutation_dicts:
    trace_name = d["trace"].split("/")[2].split(".")[0]
    if trace_name == 'WIRED_900kbps':
        #SAC, tuned
        if (d["alg"] == "SAC") and (d["tuned"] == True):
            new_permutation_dicts.append(d)

    elif trace_name == "WIRED_200kbps":
        #TD3_WIRED_200kbps_200_delay_False_norm_states_True_tuned_False
        if (d["alg"] == "TD3") and (d["tuned"] == False):
            new_permutation_dicts.append(d)

    elif trace_name == "WIRED_35mbps":
        #TD3_WIRED_35mbps_200_delay_False_norm_states_True_tuned_True
        if (d["alg"] == "TD3") and (d["tuned"] == True):
            new_permutation_dicts.append(d)

    elif trace_name == "4G_700kbps":
        #delay True is better but we go with delay False
        #TD3_4G_700kbps_200_delay_True_norm_states_True_tuned_False
        if (d["alg"] == "TD3") and (d["tuned"] == True):
            new_permutation_dicts.append(d)
    
    elif trace_name == "4G_3mbps":
        #TD3_4G_3mbps_200_delay_False_norm_states_True_tuned_True
        if (d["alg"] == "TD3") and (d["tuned"] == True):
            new_permutation_dicts.append(d)
    
    elif trace_name == "4G_500kbps":
        #TD3_4G_500kbps_200_delay_False_norm_states_True_tuned_True
        if (d["alg"] == "TD3") and (d["tuned"] == True):
            new_permutation_dicts.append(d)

    
for di in new_permutation_dicts:
    print(di)

{'trace': './traces/WIRED_900kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'SAC', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/WIRED_900kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'SAC', 'tuned': True, 'reward_profile': 1}
{'trace': './traces/WIRED_900kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'SAC', 'tuned': True, 'reward_profile': 2}
{'trace': './traces/WIRED_900kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'SAC', 'tuned': True, 'reward_profile': 3}
{'trace': './traces/WIRED_900kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'SAC', 'tuned': True, 'reward_profile': 4}
{'trace': './traces/WIRED_35mbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/WIRED_35mbps.json', 'delay_states

In [9]:
type(new_permutation_dicts[0]["reward_profile"])

int

In [11]:
len(new_permutation_dicts)

30

In [10]:
with open('permutation_dicts_reward_profiles.pkl', 'wb') as f:
    pickle.dump(new_permutation_dicts, f)

## See what is finished and what is missing

In [ ]:
data_dir_list = os.listdir("./data_mp")
output_dir_list = os.listdir("./output_mp")
tensorboard_dir_list = os.listdir("./tensorboard_logs_mp")

In [ ]:
len(data_dir_list), len(output_dir_list), len(tensorboard_dir_list)

In [ ]:
permutations_done = []
permutations_there_but_not_done = []

for d in output_dir_list:
    
    pickle_path = "output_mp/" + d
    l_rate = pd.read_pickle(pickle_path)
    trace = list(l_rate.keys())[0]
    if len(l_rate[trace]) < 10:
        print(len(l_rate[trace]))
        permutations_there_but_not_done.append(d)
    else: 
        d_splitted = d.split("_")
        alg = d_splitted[3]
        trace_name = d_splitted[4] + "_" + d_splitted[5]
        trace = "./traces/" + trace_name + ".json"
        step_time = int(d_splitted[6])
        if d_splitted[8] == "True":
            delay_states = True
        else:
            delay_states = False
        if d_splitted[11] == "True":
            normalize_states = True
        else:
            normalize_states = False
        if d_splitted[13].split(".")[0] == "True":
            tuned = True
        else:
            tuned = False

        d_final = {
         'trace': trace,
         'delay_states': delay_states,
         'normalize_states': normalize_states,
         'step_time': step_time,
         'alg': alg,
         'tuned': tuned
        }

        permutations_done.append(d_final)

In [ ]:
permutations_there_but_not_done

In [ ]:
len(permutations_done)

In [ ]:
len(permutation_dicts) - len(permutations_done)

In [ ]:
permutations_to_do = [i for i in permutation_dicts if i not in permutations_done]
len(permutations_to_do)

In [ ]:
real_permutations_to_do = []

for p in permutations_to_do:
    if (p["normalize_states"] == True) and (p["alg"] != "PPO") and (p["trace"] != './big_trace/big_trace2.json'):
        real_permutations_to_do.append(p)

In [ ]:
len(real_permutations_to_do)

In [ ]:
real_permutations_to_do

In [ ]:
with open('permutation_dicts_big_trace_only.pkl', 'wb') as f:
    pickle.dump(real_permutations_to_do, f)

In [ ]:
len(pd.read_pickle('permutation_dicts_big_trace_only.pkl'))

In [ ]:
for i in range(3,6):
    print(i)

## Deleting permutations_there_but_not_done - uncommment

In [ ]:
for fname in permutations_there_but_not_done:
    fname = "./output_mp/" + fname
    print(fname)
    if os.path.isfile(fname): # this makes the code more robust
        # os.remove(fname)
        continue

In [ ]:
# to_delete = [
# "SAC_WIRED_900kbps_400_delay_False_norm_states_True_tuned_True",
# "SAC_WIRED_900kbps_600_delay_True_norm_states_False_tuned_True",
# "PPO_WIRED_900kbps_400_delay_True_norm_states_True_tuned_False",
# "PPO_WIRED_900kbps_400_delay_False_norm_states_False_tuned_True",
# "TD3_WIRED_35mbps_200_delay_True_norm_states_True_tuned_False"  
# ]